In [ ]:
# Check if running in Google Colab
import os

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    if os.path.basename(os.getcwd()) == 'LocCCA':
        pass
    else:
        if not os.path.exists('LocCCA'):
            !git clone https://github.com/khaytao/LocCCA/
        %cd LocCCA
    # Clone repository and install dependencies if needed


    
    # Change to LocCCA directory if not already there

        
    %pip install -r requirements.txt

    

In [11]:
import json
# Load config file
with open('config/colab_configs.json', 'r') as f:
    configs = json.load(f)

    print(configs["tasks"])


['data-generation', 'train', 'eval']


In [ ]:
# Run data generation if enabled in config
if "data-generation" in configs["tasks"]:
    data_gen_args = configs["data-generation"]
    cmd = f"python dataset_generation.py"
    
    # Add all arguments from config
    cmd += f" --room_dim {data_gen_args['room_dim'][0]} {data_gen_args['room_dim'][1]} {data_gen_args['room_dim'][2]}"
    cmd += f" --t60 {data_gen_args['t60']}"
    cmd += f" --fs {data_gen_args['fs']}" 
    cmd += f" --mic_height {data_gen_args['mic_height']}"
    cmd += f" --array_distance {data_gen_args['array_distance']}"
    cmd += f" --height {data_gen_args['height']}"
    cmd += f" --num_speakers {data_gen_args['num_speakers']}"
    cmd += f" --distribution {data_gen_args['distribution']}"
    cmd += f" --centroids {data_gen_args['centroids'][0]} {data_gen_args['centroids'][1]}"
    cmd += f" --std {data_gen_args['std']}"
    cmd += f" --data_location {data_gen_args['data_location']}"
    cmd += f" --dataset_name {data_gen_args['dataset_name']}"
    cmd += f" --room_resolution {data_gen_args['room_resolution']}"
    
    !{cmd}


In [ ]:
# Run evaluation if enabled in config
if "eval" in configs["tasks"]:
    import eval_model
    from argparse import Namespace
    
    # Create args namespace from config
    eval_args = configs["eval"]
    args = Namespace(
        model=eval_args['model'],
        data_dir=eval_args['data_dir'],
        threshold=eval_args['threshold'],
        plot=eval_args['plot'],
        performance_file=eval_args['performance_file']
    )
    
    # Run evaluation
    eval_model.main(args)
    
    if IN_COLAB:
        from google.colab import files
        files.download(eval_args['performance_file'])


Figure(640x480)
Number of failures: 5/10
MSE: 0.02382950857281685

Results saved to performance.xlsx
Model: music
Failure percentage: 50.00%
MSE: 0.0238


INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []

100%|██████████| 10/10 [00:03<00:00,  3.25it/s]


In [ ]:
# Download generated data if in colab
if IN_COLAB:
    import os
    import shutil
    
    download = input("Would you like to download the generated data? (y/n): ")
    
    if download.lower() == 'y':
        # Create zip file of generated data
        data_dir = "data/generated"
        zip_name = "generated_data.zip"
        
        if os.path.exists(data_dir):
            shutil.make_archive("generated_data", 'zip', data_dir)
            files.download(zip_name)
            print(f"Downloaded {zip_name}")
        else:
            print(f"Directory {data_dir} does not exist")
